In [1]:
import tensorflow as tf
import pickle
import numpy as np
import time
import matplotlib.pyplot as plt
from load_cifar import *

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

In [2]:
tf.reset_default_graph()

<h1>Define Hyper-perparmeter</h1>

In [3]:
#learning rate
lr = 0.01
#number of traning steps
num_epochs =10

#number of batch_size
batch_size = 1024

#network parameters
num_classes = 10

<h1>Define Placeholder</h1>

In [4]:
X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name='X')
Y = tf.placeholder(tf.float32, shape=[None, num_classes], name='Y')
drop_prob = tf.placeholder(tf.float32, name='drop_prob')
learning_rate = tf.placeholder(tf.float32, shape=[])

<h1>Define Neural Network Architecture</h1>

In [5]:
def fully_connected_net(x, drop_prob):
    flat = tf.contrib.layers.flatten(x)  
    
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=4096, activation_fn=tf.nn.relu)
    full1 = tf.nn.dropout(full1, drop_prob)
    full1 = tf.layers.batch_normalization(full1)   
    
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=1000, activation_fn=tf.nn.relu)
    full2 = tf.nn.dropout(full2, drop_prob)
    full2 = tf.layers.batch_normalization(full2)

    full3 = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=4000, activation_fn=tf.nn.relu)
    full3 = tf.nn.dropout(full3, drop_prob)
    full3 = tf.layers.batch_normalization(full3)    
    
    full4 = tf.layers.dense(full3, units=1024, activation=tf.nn.relu)
    full4 = tf.nn.dropout(full4, drop_prob)
    full4 = tf.layers.batch_normalization(full4)  

    out = tf.layers.dense(full4, units=10, activation=tf.nn.softmax)
    return out


<h1>Define cost andoptimization</h1>

In [6]:
#predicted labels
logits = fully_connected_net(X,drop_prob)
#define loss

# Loss and Optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
train_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

We select the batch size of 256

In [7]:
init = tf.global_variables_initializer()
batch_size = 256
lr_error_list = []
for lr in [0.0001,0.001, 0.01]:
    with tf.Session() as sess:
        #Initialize the variables
        sess.run(init)
        #fetch batch
        for i in range(num_epochs):
            for batch_id in np.arange(1,2):
                for fearues_batch, labels_batch in load_preprocessed_training_batch(batch_id,batch_size):
                    #run optimization
                    sess.run(train_optimizer, feed_dict={X:fearues_batch, 
                                                         Y:labels_batch,
                                                         learning_rate : lr,
                                                             drop_prob: 0.8})

            acc = sess.run(accuracy,feed_dict={X:fearues_batch,
                                               Y:labels_batch,
                                               learning_rate : lr,
                                              drop_prob : 1.0})
            print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
        print("Training finished!") 
        test_features, test_labels = pickle.load(open('cifar-preprocessed/batch_test','rb'))
        acc_test = sess.run(accuracy, feed_dict={X:test_features, 
                                                       Y:test_labels,
                                                       learning_rate : lr,
                                                      drop_prob:1.0})
        print("Testing ACcuracy:", acc_test)
        lr_error_list.append(acc_test)
plt.plot([0.0001,0.001, 0.01], lr_error_list)
plt.xlabel('learning rate')
plt.ylabel('test error')
plt.show()

step 0, Accuracy= 0.188
step 1, Accuracy= 0.270
step 2, Accuracy= 0.395
step 3, Accuracy= 0.316
step 4, Accuracy= 0.344
step 5, Accuracy= 0.398
step 6, Accuracy= 0.391
step 7, Accuracy= 0.453
step 8, Accuracy= 0.344
step 9, Accuracy= 0.387
Training finished!
Testing ACcuracy: 0.3615
step 0, Accuracy= 0.121
step 1, Accuracy= 0.105
step 2, Accuracy= 0.074


KeyboardInterrupt: 

learning rate selected is 0.001

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [8]:
X_valid, Y_valid = pickle.load(open('cifar-preprocessed/batch_valid','rb'))

In [11]:
lr = 0.0001
batch_size = 64
init = tf.global_variables_initializer()

with tf.Session() as sess:
    #Initialize the variables
    sess.run(init)
    #fetch batch
    for i in range(num_epochs):
        start = time.time()
        for batch_id in np.arange(1,6):
            for fearues_batch, labels_batch in load_preprocessed_training_batch(batch_id,batch_size):
                #run optimization
                sess.run(train_optimizer, feed_dict={X:fearues_batch, 
                                                     Y:labels_batch,
                                                     learning_rate : lr,
                                                         drop_prob: 0.7})
        end= time.time()
        print("step "+str(i)+", Train Time = {:.3f}".format(end-start))
        acc = sess.run(accuracy,feed_dict={X:fearues_batch,
                                           Y:labels_batch,
                                           learning_rate : lr,
                                          drop_prob : 1.0})
        print("    Train Accuracy= {:.3f}".format(acc))
        acc = sess.run(accuracy,feed_dict={X:X_valid,
                                           Y:Y_valid,
                                           learning_rate : lr,
                                          drop_prob : 1.0})
        print("    Validation Accuracy= {:.3f}".format(acc))
    print("Training finished!") 
    test_features, test_labels = pickle.load(open('cifar-preprocessed/batch_test','rb'))
    print("Testing ACcuracy:", sess.run(accuracy, 
                                        feed_dict={X:test_features, 
                                                   Y:test_labels,
                                                   learning_rate : lr,
                                                  drop_prob:1.0}))

step 0, Train Time = 179.483
    Train Accuracy= 0.250
    Validation Accuracy= 0.266
step 1, Train Time = 178.643
    Train Accuracy= 0.266
    Validation Accuracy= 0.334
step 2, Train Time = 171.987
    Train Accuracy= 0.375
    Validation Accuracy= 0.321
step 3, Train Time = 169.269
    Train Accuracy= 0.406
    Validation Accuracy= 0.369
step 4, Train Time = 169.046
    Train Accuracy= 0.312
    Validation Accuracy= 0.356
step 5, Train Time = 169.725
    Train Accuracy= 0.438
    Validation Accuracy= 0.360
step 6, Train Time = 168.806
    Train Accuracy= 0.484
    Validation Accuracy= 0.377
step 7, Train Time = 160.692
    Train Accuracy= 0.375
    Validation Accuracy= 0.385
step 8, Train Time = 165.049
    Train Accuracy= 0.328
    Validation Accuracy= 0.388
step 9, Train Time = 174.384
    Train Accuracy= 0.500
    Validation Accuracy= 0.412
Training finished!
Testing ACcuracy: 0.4107
